In [ ]:
from estnltk.storage.postgres import PostgresStorage
import configparser
import os

config = configparser.ConfigParser()
config_file = 'config\example_configuration.ini'

file_name = os.path.abspath(os.path.expanduser(os.path.expandvars(str(config_file))))
config.read(file_name)

dbname = config['source_database']['database']
user = config['source_database']['username']
password = config['source_database']['password']
host = config['source_database']['host']
port = config['source_database']['port']
role = config['source_database']['role']
schema = config['source_database']['schema']
collection = config['source_database']['collection']


storage = PostgresStorage(host=host,
                          port=int(port),
                          dbname=dbname,
                          user=user,
                          password=password,
                          schema=schema,
                          role=role,
                          temporary=False)

display(storage)

collection = storage[collection]

collection.selected_layers = ['v171_named_entities','v172_geo_terms']

INFO:storage.py:58: connecting to host: 'postgres.keeleressursid.ee', port: 5432, dbname: 'estonian-text-corpora', user: 'rasmusm'


In [3]:
terms = []
with open('geo_terms.txt','r',encoding='UTF-8') as f:
    term = f.readline()
    while term is not '':
        terms.append(term.strip())
        term = f.readline()

In [45]:
# Second storage to keep the temporary lists used for sampling

# load configuration

import configparser
import os

config = configparser.ConfigParser()
config_file = 'config\example_configuration.ini'

file_name = os.path.abspath(os.path.expanduser(os.path.expandvars(str(config_file))))

if not os.path.exists(file_name):
    raise ValueError("File {file} does not exist".format(file=str(config_file)))

if len(config.read(file_name)) != 1:
    raise ValueError("File {file} is not accessible or is not in valid INI format".format(file=config_file))

for option in ["host", "port", "database", "username", "password", "schema", "collection"]:
    if not config.has_option('target_database', option):
        prelude = "Error in file {}\n".format(file_name) if len(file_name) > 0 else ""
        raise ValueError(
            "{prelude}Missing option {option} in the section [{section}]".format(
                prelude=prelude, option=option, section='target_database'
            )
        )

config.read(file_name)

# connect to database

from estnltk.storage.postgres import PostgresStorage

dbname = config['target_database']['database']
user = config['target_database']['username']
password = config['target_database']['password']
host = config['target_database']['host']
port = config['target_database']['port']
schema = config['target_database']['schema']

localstorage = PostgresStorage(host=host,
                          port=int(port),
                          dbname=dbname,
                          user=user,
                          password=password,
                          schema=schema,
                          role=None,
                          temporary=False)

INFO:storage.py:58: connecting to host: 'localhost', port: 5432, dbname: 'ner_test', user: 'postgres'
INFO:storage.py:108: schema: 'my_schema', temporary: False, role: 'postgres'


In [46]:
from span_sampler import SpanSampler

sampler = SpanSampler(storage=localstorage,collection=collection, layer='v172_geo_terms',attribute='partofspeech')

In [49]:
from copy import copy
from estnltk.converters.label_studio.label_studio import LabelStudioExporter

for attribute in ('A','C','D','G','H','I','J','K','N','O','P','S','U','V','X','Y','Z'):
    samples = sampler(count=1000,attribute="('"+attribute+"')")

    for text, sample_span in samples:
        spanstart = sample_span[1]
        spanend = sample_span[2]
        for span in copy(text.v172_geo_terms.spans):
            if span.start != spanstart and span.end != spanend:
                text.v172_geo_terms.remove_span(span)
    
    exporter = LabelStudioExporter("pos_"+attribute+"_1000.json",['v172_geo_terms'],checkbox=True)

    only_texts = [sample[0] for sample in samples]

    exporter.convert(only_texts,append=False)

In [60]:
exporter = LabelStudioExporter("pos_"+attribute+"_1000.json",['v172_geo_terms'],checkbox=True)
print(exporter.interface_generator())


        <View>
            <Labels name="label" toName="text">
	<Label value="v172_geo_terms" background="#2497B0"/> 

            </Labels>
        <Text name="text" value="$text"/>
            
            </View>
